In [2]:
import torch
from skimage import io, transform
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

/cvhci/temp/prgc/anaconda3/envs/env/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
import torchvision

In [4]:
def build_transform(rescale_size=512, crop_size=448):
    train_transform = torchvision.transforms.Compose([
        # torchvision.transforms.Resize(size=rescale_size),
        torchvision.transforms.RandomHorizontalFlip(),
        # torchvision.transforms.RandomCrop(size=crop_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])
    test_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(size=rescale_size),
        torchvision.transforms.CenterCrop(size=crop_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])
    train_transform_strong_aug = torchvision.transforms.Compose([
        torchvision.transforms.Resize(size=rescale_size),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomCrop(size=crop_size),
        # RandAugment(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])
    return {'train': train_transform, 'test': test_transform, 'train_strong_aug': train_transform_strong_aug}

In [5]:
from PIL import Image

In [6]:
class CustomDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, mode, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return 20

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # sample = np.ones((1, 2, 3, 4))
        imarray = np.random.rand(600,600,3) * 255
        sample = Image.fromarray(imarray.astype('uint8')).convert('RGB')
        label = "This is label"

        if self.transform:
            sample = self.transform(sample)

        sample = {'data': sample, 'label': label, 'idx': idx}
        return sample

In [7]:
class CLDataTransform(object):
    def __init__(self, transform_weak, transform_strong):
        self.transform_weak = transform_weak
        self.transform_strong = transform_strong

    def __call__(self, sample):
        x_w1 = self.transform_weak(sample)
        x_w2 = self.transform_weak(sample)
        x_s = self.transform_strong(sample)
        return x_w1, x_w2, x_s

In [8]:
def build_dataset(mode, transform):
    if mode == 'train':
        dataset = CustomDataset('train', transform=transform)
    elif mode == 'val' or mode == 'test':
        dataset = CustomDataset('test', transform=transform)
    return dataset

In [9]:
def build_dataset_loader():
    transform = build_transform()
    train_dataset = build_dataset('train', CLDataTransform(transform['train'], transform['train_strong_aug']))
    test_dataset = build_dataset('test', transform['test'])
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=False)
    dataset = {'train': train_dataset, 'test': test_dataset}
    return dataset, train_loader, test_loader

In [10]:
dataset, train_loader, test_loader = build_dataset_loader()

In [11]:
from tqdm import tqdm

In [17]:
pbar = tqdm(train_loader, ncols=150, ascii=' >', leave=False, desc='training')
for it, sample in enumerate(pbar):
    index = sample['idx']
    label = sample['label']
    x, x_w, x_s = sample['data']
    assert type(sample['data']) == list and len(sample['data']) == 3
    print("The type of x is: ", sample['data'][0].shape)
    print("The type of x is: ", x[0].shape)
    print("The type of x_w is: ", x_w[0].shape)
    print("The type of x_s is: ", x_s[0].shape)
    print("The label is: ", label[0])
    break

The type of x is:  torch.Size([16, 3, 600, 600])
The type of x is:  torch.Size([3, 600, 600])
The type of x_w is:  torch.Size([3, 600, 600])
The type of x_s is:  torch.Size([3, 448, 448])
The label is:  This is label


In [1]:
import torch

In [62]:
labels = torch.Tensor([[0, 0,0,1,0,0], [0, 0,0,0,0,1], [0, 1,0,0,0,0], [0, 0,0,0,0,1], [0, 1,0,0,0,0], [0, 0,0,0,0,1], [0, 0,1,0,0,0], [0, 0,1,0,0,0]])

In [31]:
labels.shape

torch.Size([8, 6])

In [9]:
x = torch.zeros((2, 3, 16, 224, 224))
x_mo = torch.zeros((1, 25088, 96))

In [2]:
type_prob = torch.randn((4, 10, 3))

In [6]:
z = type_prob.softmax(dim=1)

In [7]:
z.shape

torch.Size([4, 10, 3])

In [4]:
x = type_prob.view(type_prob.shape[0], -1)
x.shape

torch.Size([4, 30])

In [5]:
y = x.softmax(dim=1)
y.shape

torch.Size([4, 30])

In [5]:
type_target = torch.nn.functional.one_hot(type_prob.max(dim=1)[1], 3)

In [59]:
def mixup_target(target, num_classes, lam=1.0, smoothing=0.0):
    """
    This function converts target class indices to one-hot vectors, given the
    number of classes.
    Args:
        targets (loader): Class labels.
        num_classes (int): Total number of classes.
        lam (float): lamba value for mixup/cutmix.
        smoothing (float): Label smoothing value.
    """
    off_value = smoothing / num_classes
    on_value = 1.0 - smoothing + off_value
    target1 = convert_to_one_hot(
        target,
        num_classes,
        on_value=on_value,
        off_value=off_value,
    )
    target2 = convert_to_one_hot(
        target.flip(0),
        num_classes,
        on_value=on_value,
        off_value=off_value,
    )
    return target1 * lam + target2 * (1.0 - lam)

In [58]:
def convert_to_one_hot(targets, num_classes, on_value=1.0, off_value=0.0):
    """
    This function converts target class indices to one-hot vectors, given the
    number of classes.
    Args:
        targets (loader): Class labels.
        num_classes (int): Total number of classes.
        on_value (float): Target Value for ground truth class.
        off_value (float): Target Value for other classes.This value is used for
            label smoothing.
    """

    targets = targets.long().view(-1, 1)
    return torch.full(
        (targets.size()[0], num_classes), off_value, device=targets.device
    ).scatter_(1, targets, on_value)

In [67]:
labels.shape

torch.Size([8, 6])

In [69]:
target = mixup_target(labels, 6, smoothing=0.1)

In [70]:
target.shape

torch.Size([48, 6])

In [2]:
import torch

In [25]:
x = torch.randn((4,6))
x

tensor([[ 0.6932,  0.3799,  2.0800, -0.8293, -0.1116, -0.4470],
        [ 0.1443,  0.9974,  0.8769, -0.8658,  0.0143, -0.6859],
        [-0.3839,  0.1288, -1.4223,  1.7675, -1.2043, -0.6170],
        [ 0.4345, -0.4345,  1.5307,  0.3862, -0.1115,  0.2062]])

In [26]:
z = x.softmax(dim=1)
z

tensor([[0.1488, 0.1088, 0.5957, 0.0325, 0.0666, 0.0476],
        [0.1407, 0.3303, 0.2928, 0.0513, 0.1236, 0.0614],
        [0.0778, 0.1299, 0.0275, 0.6689, 0.0343, 0.0616],
        [0.1484, 0.0622, 0.4440, 0.1414, 0.0859, 0.1181]])

In [12]:
a = torch.tensor([[0,0,0,1,0,0], [0,1,0,0,0,0], [0,1,0,0,0,0], [1,0,0,0,0,0]])

In [9]:
a.shape

torch.Size([4, 6])

In [6]:
def get_smoothed_label_distribution(labels, num_class, epsilon):
    smoothed_label = torch.full(size=(labels.size(0), num_class), fill_value=epsilon / (num_class))
    addier = torch.mul(labels.cpu(), 1 - epsilon)
    smoothed_label = torch.add(smoothed_label, addier)
    # smoothed_label.scatter_(dim=1, index=torch.unsqueeze(labels, dim=1).cpu(), value=1 - epsilon)
    return smoothed_label

In [15]:
smoothed = get_smoothed_label_distribution(a, 6, 0.6)
smoothed

tensor([[0.1000, 0.1000, 0.1000, 0.5000, 0.1000, 0.1000],
        [0.1000, 0.5000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.5000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.5000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000]])

In [7]:
def kl_div(p, q, base=2):
    # p, q is in shape (batch_size, n_classes)
    if base == 2:
        return (p * p.log2() - p * q.log2()).sum(dim=1)
    else:
        return (p * p.log() - p * q.log()).sum(dim=1)


def symmetric_kl_div(p, q, base=2):
    return kl_div(p, q, base) + kl_div(q, p, base)


def js_div(p, q, base=2):
    # Jensen-Shannon divergence, value is in (0, 1)
    m = 0.5 * (p + q)
    return 0.5 * kl_div(p, m, base) + 0.5 * kl_div(q, m, base)

In [8]:
def clean_probs(probs, given_labels): 
    return (1 - js_div(probs, given_labels))

In [27]:
result = clean_probs(z, get_smoothed_label_distribution(a, 6, 0.6))
result

tensor([0.6678, 0.9392, 0.7140, 0.8359])

In [23]:
result = clean_probs(z, get_smoothed_label_distribution(a, 6, 0.1))
result

tensor([0.5712, 0.4096, 0.8021, 0.3492])

In [24]:
result = clean_probs(z, get_smoothed_label_distribution(a, 6, 0.01))
result

tensor([0.4355, 0.2667, 0.6857, 0.1941])

In [1]:
import torch

In [5]:
thresh = 0.1

In [3]:
x = torch.rand((2, 4))

In [4]:
y = torch.rand((2, 4))

In [7]:
x * y

tensor([[0.0184, 0.6524, 0.1635, 0.0159],
        [0.2061, 0.3924, 0.4875, 0.5115]])

In [6]:
mask = (torch.abs(x * y) >= thresh).type(torch.FloatTensor)
mask

tensor([[0., 1., 1., 0.],
        [1., 1., 1., 1.]])

In [9]:
import numpy as np

In [15]:
clip_narry = np.linspace(1-0.2, 1, num=2)
clip_narry

array([0.8, 1. ])

In [11]:
clip_narry = clip_narry[::-1]

In [12]:
clip_narry

array([1.        , 0.98947368, 0.97894737, 0.96842105, 0.95789474,
       0.94736842, 0.93684211, 0.92631579, 0.91578947, 0.90526316,
       0.89473684, 0.88421053, 0.87368421, 0.86315789, 0.85263158,
       0.84210526, 0.83157895, 0.82105263, 0.81052632, 0.8       ])

In [16]:
z = round(26 * 0.1)
z

3